# First jupyter notebook

This is a jupyer notebook plain text

## subhead

```python
import this

print('\n'.join(x for x in 'hello world'))
```

<table><tr><th>A</th><th>B</th><th>C</th></tr>
    <tr><td>foo</td><td>bar</td><td>baz</td></tr>
    <tr><td>foo</td><td>bar</td><td>baz</td></tr>
    <tr><td>foo</td><td>bar</td><td>baz</td></tr>
</table>


<div style="border: 2px solid black;width:20%;margin:auto">
<img src="https://c.pxhere.com/photos/b4/40/puppy_dog_pet_animal_cute_white_adorable_canine-1264875.jpg!d" alt="Cute puppy">
</div>

In [1]:
import json
import altair as alt
import pandas as pd
from altair.vega import v3

from IPython.display import HTML
from altair.utils.mimebundle import spec_to_mimebundle

#def colab_renderer(spec, **metadata):
#    return spec_to_mimebundle(spec, format='html',
#                              mode='vega',
#                             vega_version=v3.SCHEMA_VERSION,
#                              vegaembed_version='3.14',
#                              **metadata)
 

#v3.renderers.register('colab', colab_renderer)
#v3.renderers.enable('notebook')
#alt.renderers.enable('notebook')

In [2]:
# Define the function for rendering
def add_autoincrement(render_func):
    # Keep track of unique <div/> IDs
    cache = {}
    def wrapped(chart, id="vega-chart", autoincrement=True):
        """Render an altair chart directly via javascript.
        
        This is a workaround for functioning export to HTML.
        (It probably messes up other ways to export.) It will
        cache and autoincrement the ID suffixed with a
        number (e.g. vega-chart-1) so you don't have to deal
        with that.
        """
        if autoincrement:
            if id in cache:
                counter = 1 + cache[id]
                cache[id] = counter
            else:
                cache[id] = 0
            actual_id = id if cache[id] == 0 else id + '-' + str(cache[id])
        else:
            if id not in cache:
                cache[id] = 0
            actual_id = id
        return render_func(chart, id=actual_id)
    # Cache will stay defined and keep track of the unique div Ids
    return wrapped


@add_autoincrement
def render(chart, id="vega-chart"):
    # This below is the javascript to make the chart directly using vegaEmbed
    chart_str = """
    <div id="{id}"></div><script>
    //require(["vega-embed"], function(vegaEmbed) {{
    (function(){{
        const spec = {chart};     
        vegaEmbed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
    }})();
    //}});
    </script>
    """
    return HTML(
        chart_str.format(
            id=id,
            chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
        )
    )


In [3]:
# Create the correct URLs for require.js to find the Javascript libraries
vega_url = 'https://cdn.jsdelivr.net/npm/vega@' + v3.SCHEMA_VERSION
vega_lib_url = 'https://cdn.jsdelivr.net/npm/vega-lib'
vega_lite_url = 'https://cdn.jsdelivr.net/npm/vega-lite@' + alt.SCHEMA_VERSION
vega_embed_url = 'https://cdn.jsdelivr.net/npm/vega-embed@3'
noext = "?noext"

#paths = {
#    'vega': vega_url + noext,
#    'vega-lib': vega_lib_url + noext,
#    'vega-lite': vega_lite_url + noext,
#    'vega-embed': vega_embed_url + noext
#}
paths = [vega_url, vega_lite_url, vega_embed_url]

workaround = """
//requirejs.config({{
//    baseUrl: 'https://cdn.jsdelivr.net/npm/',
//    paths: {paths}
//}});
const paths = {paths};
paths.forEach(function(path){{
  var script = document.createElement('script');
  script.setAttribute('src', path);
    document.head.append(script);
}});
"""

HTML("".join((
    "<script>",
    workaround.format(paths=json.dumps(paths)),
    "</script>",
    "This code block sets up embedded rendering in HTML."
)))

In [4]:
df = pd.DataFrame(dict(a=tuple("howdy"), b=[1,2,3,4,5]))
render(alt.Chart(df, width=360).mark_bar().encode(x='a:O', y='b:Q', tooltip='b'))

In [5]:
def Data(df, name="data"):
    return [dict(name=name, values=df.to_dict(orient="records"))]


def word_cloud(data, pixwidth=800, pixheight=400):
    wordcloud = {
        "$schema": "https://vega.github.io/schema/vega/v3.json",
        "width": pixwidth,
        "height": pixheight,
        "padding": 0,
        "data": Data(df, name="words")
    }
    scale = dict(
        name="color",
        type="ordinal",
        range=["cadetblue", "royalblue", "steelblue", "navy", "teal"]
    )
    mark = {
        "type":"text",
        "from":dict(data="words"),
        "encode":dict(
            enter=dict(
                text=dict(field="word"),
                align=dict(value="center"),
                fill=dict(scale="color", field="etc"),
                hover=dict(fillOpacity=dict(value=.5)),
                update=dict(fillOpacity=dict(value=1)),
                tooltip=dict(field="etc", type="nominal", fontSize=32)
            )
        ),
        "transform": [{
            "type": "wordcloud",
            "text": dict(field="word"),
            "size": [pixwidth, pixheight],
            "fontSize": dict(field="datum.count"),
            "fontSizeRange": [20, 80],
            "padding": 3
        }]
    }
    wordcloud["scales"] = [scale]
    wordcloud["marks"] = [mark]
    return wordcloud

In [6]:
df = pd.DataFrame(dict(word=('foo', 'bar', 'baz'),
                       count=(1,3,5),
                      etc=('label', 'another', 'third')))
#v3.vega(word_cloud(df))
render(word_cloud(df))